In [1]:
# importing library to be used

import os
from fpdf import FPDF
import cv2
import pytesseract

Executable = "ffmpeg.exe"
input = "VideoLecture4.mp4 "
if not(os.path.exists("C:/Users/HP User/Desktop/ML Workshop/img")):
    os.mkdir("C:/Users/HP User/Desktop/ML Workshop/img")

output="img/image%d.png"

# ffmpeg command for getting images from video

cmd=Executable + " -i "+ input + "-filter_complex \"select='gt(scene,0.05)',metadata=print:file=time.txt\" -vsync vfr " + output 
os.system(cmd)

imagelist=os.listdir("C:/Users/HP User/Desktop/ML Workshop/img")

# imagelist is the list with all image filenames
for image in imagelist:
    img = cv2.imread("C:/Users/HP User/Desktop/ML Workshop/img/"+image)
    if img is not None:
        detector = cv2.CascadeClassifier("haarcascade_frontalface_alt (1).xml")
        faces=detector.detectMultiScale(img)
        pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
        text = pytesseract.image_to_string(img, config='')
        if (len(text)<5) | (len(faces)>0) :
            os.remove("C:/Users/HP User/Desktop/ML Workshop/img/"+image)

            
pdf = FPDF()
imagelist=os.listdir("C:/Users/HP User/Desktop/ML Workshop/img")
for image in imagelist:
    pdf.add_page()
    pdf.image("C:/Users/HP User/Desktop/ML Workshop/img/"+image)

pdf.output("Slides.pdf", "F")

''

In [2]:
# importing libraries required for text transcript conversion

import subprocess
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [3]:
# getting audio file 

command = 'ffmpeg -i VideoLecture4.mp4 -ab 160k -ar 44100 -vn audio.wav'
subprocess.call(command, shell=True)

0

In [4]:
# credentials for connecting with IBM Watson


apikey = 'TegrIibKeT6J3KaqEyDUjWd8jEURt3iTr7EQp7Y1x8TU'
url = 'https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/e3accf22-7aea-44ff-8b8e-f8d354183b17'

In [5]:
#calling speech to text service of IBM Watson

authenticator = IAMAuthenticator(apikey)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)

In [6]:
#passing audio input to service 

with open('audio.wav', 'rb') as f:
    res = stt.recognize(audio=f, content_type='audio/wav', model='en-AU_NarrowbandModel', continuous=True).get_result()

In [7]:
text = [result['alternatives'][0]['transcript'].rstrip() + '.' for result in res['results']]

In [8]:
#making .txt file from extracted text

text = [para[0].title() + para[1:] for para in text]
transcript = ''.join(text)
with open('output.txt', 'w') as out:
    out.writelines(transcript)

In [ ]:
transcript